In [1]:
!pip install bertopic
!pip install safetensors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 5.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)

# Prototyping

In [7]:
from bertopic import BERTopic
topic_model = BERTopic.load("MaartenGr/BERTopic_Wikipedia")

In [8]:
compact_topics = topic_model.generate_topic_labels(nr_words=3, word_length=15)

In [ ]:
compact_topics[pred[0][0]+1]

'1457_presidents_presidential_presidency'

In [9]:
from utils.bench_ls_data_provider import BenchLSDataProvider

In [10]:
dp = BenchLSDataProvider()
data = dp.provide_data_as_numpy_array()

In [11]:
original_sentences = [sample[0] for sample in data]

In [16]:
topics_idx = topic_model.transform(original_sentences)

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

2024-07-08 12:07:47,664 - BERTopic - Predicting topic assignments through cosine similarity of topic and document embeddings.


In [13]:
topics_idx = [topic_idx+1 for topic_idx in topics_idx[0]]

In [18]:
topics_idx[0]

array([ 847,  996,  127,  789,  266, 2178,    5, 1474, 1276,  960,  205,
       1345,   22,  266,  266,   40,  252, 1326,  528,  266, 1836,  666,
         45,   40, 2094,   27,    4,  582,  645,  979,   15, 1760, 1147,
         57,   17,  215,  140, 1865,  967,  195, 1847, 1833,   57,   47,
       1970, 1960,  179,   42, 1281,  987,  391,  113, 1273, 1381, 1184,
        105, 1795,  166,  941,  266, 1317,  433, 2169, 2278,  102,  809,
        398,  915, 1396, 1494, 1255, 2234, 2212,  451, 1302,    2, 2210,
       1874,  835, 1936, 1101, 1281,  613, 1775,  125, 1138,  238, 1913,
       1623,  679,  662,    9, 1468,   57,   94,  987, 1795, 1314,  332,
       1661,  178,    7, 1668,   40,  536,    0, 1183,  543,  223,  102,
       1515,  686,   79, 1913, 2022,   95,   17, 1700, 1605,   57,  550,
          2,  644, 2334,  991,  191,  355,  684,  557, 2160,  446,  714,
        266, 1382,   17, 1927, 2094, 2289,   15,  278,   17,    5,  161,
       1967, 1518,  523,   46, 1635,   61, 1588, 17

# Demo on EN datasets with SimpleBERTLexicalSimplifier

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import sys
sys.path.append('/content/drive/MyDrive/nlp_ss24/multilingual-lexical-simplification')
sys.path.append('/content/drive/MyDrive/nlp_ss24/multilingual-lexical-simplification/src')

In [3]:
from llm_lexical_simplifier import LLMLexicalSimplifier
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForMaskedLM
from simple_bert_lexical_simplifier import SimpleBertLexicalSimplifier
from benchmark_suite import BenchmarkSuite
from language import Language

Using cuda for model inference.


In [4]:
bert_tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")
bert_model = AutoModelForMaskedLM.from_pretrained("google-bert/bert-base-cased")

Some weights of the model checkpoint at google-bert/bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
bert_ls = SimpleBertLexicalSimplifier(model=bert_model,
                                      tokenizer=bert_tokenizer,
                                      mask_token='[MASK]',
                                      pattern='{original_sentence}. The simpler version of the previous sentence in the domain "{topic}" is: {sentence_with_complex_word_masked}. Top ten results:',
                                      exemplars=None)

Using mask token: "[MASK]".


In [6]:
suite = BenchmarkSuite(bert_ls,
                       {Language.EN: {'pattern': '{original_sentence}. The simpler version of the previous sentence in the domain "{topic}" is: {sentence_with_complex_word_masked}. Top ten results:', 'exemplars': None}}, should_pass_topic=True)
suite.run()

Benchmarking model on EN ...
Benchmarking model on BenchLSDataProvider...


Batches:   0%|          | 0/30 [00:00<?, ?it/s]

2024-07-08 12:11:56,708 - BERTopic - Predicting topic assignments through cosine similarity of topic and document embeddings.
Benchmarking: 100%|██████████| 929/929 [00:10<00:00, 88.11it/s]

Benchmarking model on LexMTurkDataProvider...


Batches:   0%|          | 0/16 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

2024-07-08 12:12:13,956 - BERTopic - Predicting topic assignments through cosine similarity of topic and document embeddings.
Benchmarking: 100%|██████████| 239/239 [00:02<00:00, 88.73it/s]


Benchmarking model on TsarENDataProvider...


Batches:   0%|          | 0/13 [00:00<?, ?it/s]

2024-07-08 12:12:16,922 - BERTopic - Predicting topic assignments through cosine similarity of topic and document embeddings.
Benchmarking: 100%|██████████| 386/386 [00:04<00:00, 79.18it/s]
